In [16]:
import pandas as pd
import plotnine as p
import anndata as ad
import scanpy as sc
import numpy as np
import matplotlib as mpl

In [17]:
df = pd.read_csv(
    "../../05_scafe/data/count_data.tsv",
    sep="\t",
    header=None,
    names=["project", "sample", "ctss", "barcode"],
)

In [18]:
ahca = pd.read_csv("../data/ahca.txt", header=None, sep="\t", names=["id", "organ"])
ahca = dict(zip(ahca["id"], ahca["organ"]))
df["sample_fix"] = df["sample"].replace(ahca)
df["id"] = df["project"] + "." + df["sample"]
df["id_fix"] = df["project"] + "." + df["sample_fix"]

In [23]:
obs = pd.read_csv(
    "../../04_subcluster_v2/data/03_subcluster.tsv",
    sep="\t",
    index_col=0,
    low_memory=False,
)

In [24]:
x = obs.dtypes
x = x[x == "object"].index
obs[x] = obs[x].astype("string")
obs["id"] = obs["project"] + "." + obs["sample"]
df = df.loc[df["id_fix"].isin(obs["id"].unique())]

In [ ]:
adatas = []
for i in range(df.shape[0]):
    print(i)
    filen = df.iloc[i]["id"]
    adata = sc.read_10x_mtx(f"../../05_scafe/scafe/merge_all/count/{filen}/matrix/")
    adata.obs_names = df.iloc[i]["sample_fix"] + "_" + adata.obs_names.str.strip("-1")
    adatas.append(adata)

In [32]:
adata = ad.concat(adatas)

In [33]:
adata = adata[adata.obs.index.isin(obs.index), :]

In [34]:
adata.obs = pd.merge(adata.obs, obs, left_index=True, right_index=True)

In [ ]:
adata.write("../data/01_load_CRE.h5ad")